<a href="https://colab.research.google.com/github/maengg/Kaggle_Kernel_Study/blob/main/Porto_Data_Preparation_%26_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***운전자가 내년에 보험금을 청구할 것인지 예측.***

평가 지표 : Normalized Gini Coefficient

***gini = 2 X AUC-1*** 이다. 즉, 모델이 랜덤하게 찍는 경우 AUC는 0.5지만, 지니 계수를 사용하면 0이 나온다. (보기 더 좋으니까 AUC 대신 사용하는 거 아니겠냐는 말이 있다.)

ROC AUC Curve는 threshold를 0~1까지 다 따지는 것. threshold 값 별로 재현율과 위양성률이 어떻게 되는 지 점으로 찍은 후 쭉 그리면 Curve가 나온다. 이때 이 Curve 밑 면적이 score가 된다.

즉, 지니 계수는 ROC AUC Score와 거의 비슷한 목적으로 쓰인다. (랜덤으로 찍는 경우 - 0, 퍼펙트 - 0.5)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# null 채우는 것.
from sklearn.impute import SimpleImputer
# 중요한 numerical 특성들을 곱하는 것.
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.preprocessing import StandardScaler
# 중요한 특성들만 뽑기 위한 것.
from sklearn.feature_selection import VarianceThreshold # 특성의 분산이 작다 -> 정보가 작다. 스펙트럼이 넓어야 얻을 수 있는 정보가 많다. 0과 1인데 99%가 1이면 별로 얻을 정보가 없다.(분산이 작다.)
# 특성의 중요도에 따라 Selection 한다.
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

In [26]:
NROWS = 50000
train = pd.read_csv('/content/drive/MyDrive/train.csv', nrows=NROWS)
test = pd.read_csv('/content/drive/MyDrive/test.csv', nrows=NROWS)

In [27]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [28]:
cat_cols = [col for col in train.columns if 'cat' in col]

In [29]:
# 카테고리컬 특성에서 카디날리티 확인.
for col in cat_cols:
  print(col, train[col].nunique())

ps_ind_02_cat 5
ps_ind_04_cat 3
ps_ind_05_cat 8
ps_car_01_cat 13
ps_car_02_cat 2
ps_car_03_cat 3
ps_car_04_cat 10
ps_car_05_cat 3
ps_car_06_cat 18
ps_car_07_cat 3
ps_car_08_cat 2
ps_car_09_cat 6
ps_car_10_cat 3
ps_car_11_cat 104


In [30]:
train.shape, test.shape

((50000, 59), (50000, 58))

In [31]:
# 중복 없음.
train.drop_duplicates()
train.shape

(50000, 59)

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 59 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              50000 non-null  int64  
 1   target          50000 non-null  int64  
 2   ps_ind_01       50000 non-null  int64  
 3   ps_ind_02_cat   50000 non-null  int64  
 4   ps_ind_03       50000 non-null  int64  
 5   ps_ind_04_cat   50000 non-null  int64  
 6   ps_ind_05_cat   50000 non-null  int64  
 7   ps_ind_06_bin   50000 non-null  int64  
 8   ps_ind_07_bin   50000 non-null  int64  
 9   ps_ind_08_bin   50000 non-null  int64  
 10  ps_ind_09_bin   50000 non-null  int64  
 11  ps_ind_10_bin   50000 non-null  int64  
 12  ps_ind_11_bin   50000 non-null  int64  
 13  ps_ind_12_bin   50000 non-null  int64  
 14  ps_ind_13_bin   50000 non-null  int64  
 15  ps_ind_14       50000 non-null  int64  
 16  ps_ind_15       50000 non-null  int64  
 17  ps_ind_16_bin   50000 non-null 